In [153]:
import sympy
import pandas as pd
from symcircuit.symbolic_solver.domains import ExperimentResult

# Loading the Results of VLSI CMMF Exploration

In [2]:
results = ExperimentResult("VLSI_CMMF_POST_EXPERIMENT")
dir_of_results = results.find_results_file("Runs/VLSI_CMMF_Automated_NA")
results = results.load(f"{dir_of_results[0]}/results.pkl")

In [3]:
results.classifications_dict.keys()

dict_keys(['Z1_Z2_Z3_Z4_Z5_Z6', 'Z2_Z3_Z4_Z5_Z6', 'Z1_Z3_Z4_Z5_Z6', 'Z1_Z2_Z3_Z5_Z6', 'Z3_Z4_Z5_Z6', 'Z2_Z3_Z5_Z6', 'Z1_Z3_Z5_Z6', 'Z3_Z5_Z6'])

In [4]:
classifications_of_intrest = results.classifications_dict["Z1_Z2_Z3_Z4_Z5_Z6"]
len(classifications_of_intrest), classifications_of_intrest[0]

(4096,
 FilterClassification(valid=False, fType=X-INVALID-ORDER, parameters=None)zCombo=(R_1, R_2, R_3, R_4, R_5, R_6), transferFunc=R_1*R_2*R_4*R_6/(R_1*R_4*R_5 - R_2*R_3*R_4 + R_2*R_3*R_5 + R_2*R_4*R_5 + R_3*R_4*R_5), )

In [5]:
# df = results.flatten_classifications() # This way we can access all the classsifcation into a pandas df

# Examining the Higher Order Filters 

In [21]:
classifications_of_intrest = []

for key in results.classifications_dict.keys():
    classifications = results.classifications_dict[key]

    classifications_of_intrest += ([classification 
                                for classification in classifications 
                                if (classification.fType in ["X-INVALID-ORDER", "X-INVALID-NUMER"] )]
                                )
    
    fTypes = set(classification.fType for classification in classifications_of_intrest)

len(classifications_of_intrest), fTypes

(6779, {'X-INVALID-NUMER', 'X-INVALID-ORDER'})

## Update filterOrders

In [24]:
count = {
    "X-CONST" : 0,
}
s = sympy.symbols("s")

for i, classification in enumerate(classifications_of_intrest):
    tf = classification.transferFunc

    if not (s in tf.free_symbols): # TF id NOT a function of S - Pure amplifier/level-shifter
        ftype = "X-CONST"
        count[ftype] += 1
        classification.fType = "X-CONST"
        classification.filterOrder = "Zero"

count

{'X-CONST': 108}

## Load the old classification into a classifier object

In [150]:
from symcircuit.symbolic_solver.filter import Filter_Classifier

classifier = Filter_Classifier()
classifier.overwrite_classifications(classifications_of_intrest)
classifier.isClassified(), len(classifier.classifications)


2025-01-20 15:00:54,079 - symcircuit.symbolic_solver.filter - !!!! Clearning the filter classifications !!!!


(True, 6779)

In [12]:
# classifier.classifyFilter(filterOrder="FirstOrder")

In [13]:
# classifier.summarizeFilterType()

# Analyze higher orders
- **LP-3rd order**: num_deg = 0, denom_deg = 3
    -  The denom is decomposable into 1 second-orders and 1 first-order
- **HP-4th Order**: num_deg = 4, denom_deg = 4
    -  N(s) has only s^4 term
- **BP-4th order**: num_deg = 0, denom_deg = 4
    -  N(s) only has s^2 term
    -  The denom is decomposable into 2 second-orders

In [118]:
def is_poly_stable(poly: sympy.Poly) -> bool:

    if poly.degree() == 2:
        a2 = poly.as_dict().get((2,), None)
        a1 = poly.as_dict().get((1,), None)
        if a1 is None or a2 is None:
            raise ValueError(f"Invalid second order poly: a2 = {a2} a1 = {a1} in {poly}")
        if sympy.ask(sympy.Q.negative(a2*a1)):
            return False
        
    elif poly.degree() == 1:
        a1 = poly.as_dict().get((1,), None)
        a0 = poly.as_dict().get((0,), None)
        if a1 is None or a0 is None:
            raise ValueError(f"Invalid first order poly: a2 = {a1} a1 = {a0} in {poly}")
        
        if sympy.ask(sympy.Q.negative(a1*a0)):
            return False

    else:
        raise ValueError(f"Unsupported poly order: {poly.degree()}")
    
    return True

### LP-3rd Order

In [163]:
from tqdm import tqdm
# from tqdm.gui import tqdm

def get_3rd_order_lp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=3, numer_order=0, printMessage=False)
    print(f"{count} candidates for 3rd-order LP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break
        gain = None
        if valid:
            count_valid += 1
            tf = classification.transferFunc
            gain = sympy.limit(tf, sympy.symbols("s"), 0)
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "tf" : sympy.latex(classification.transferFunc),
            "DC-Gain": gain,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output
output_lp = get_3rd_order_lp(classifier)

df_lp = pd.DataFrame(output_lp)
df_lp.to_csv(f"Runs/{results.experiment_name}/3rd_LP.csv")
df_lp

132 candidates for 3rd-order LP


100%|██████████| 132/132 [00:08<00:00, 15.83it/s]


44 verified filters


,valid,zCombo,tf,DC-Gain,k,numer,denom,num-factor-count,denom-factor-count
0,False,"(1/(C_1*s), R_2, R_3, 1/(C_4*s), R_5, 1/(C_6*s))",\frac{R_{2}}{C_{1} C_{4} C_{6} R_{2} R_{3} R_{...,None,R_2/C_6,[],"[Poly(s, s, domain='ZZ'), Poly(s**2 + (-R_2*R_...",0,2
1,True,"(1/(C_1*s), R_2, R_3, 1/(C_4*s), R_5, R_6/(C_6...",\frac{R_{2} R_{6}}{C_{1} C_{4} C_{6} R_{2} R_{...,R_2*R_6/R_5,R_2*R_6,[],"[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",0,2
2,False,"(1/(C_1*s), R_2, R_3, 1/(C_4*s), R_5 + 1/(C_5*...",\frac{C_{5} R_{2}}{C_{1} C_{4} C_{5} C_{6} R_{...,None,C_5*R_2/C_6,[],[Poly(s**3 + (C_4*R_2*R_3 - C_5*R_2*R_3 + C_5*...,0,1
3,False,"(1/(C_1*s), R_2, R_3, R_4/(C_4*R_4*s + 1), R_5...",\frac{R_{2} R_{4}}{C_{1} C_{4} C_{6} R_{2} R_{...,None,R_2*R_4/C_6,[],"[Poly(s, s, domain='ZZ'), Poly(s**2 + (-R_2*R_...",0,2
4,True,"(1/(C_1*s), R_2, R_3, R_4/(C_4*R_4*s + 1), R_5...",\frac{R_{2} R_{4} R_{6}}{C_{1} C_{4} C_{6} R_{...,R_2*R_6/R_5,R_2*R_4*R_6,[],"[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",0,2
...,...,...,...,...,...,...,...,...,...
127,False,"(R_1/(C_1*R_1*s + 1), 1/(C_2*s), R_3 + 1/(C_3*...",\frac{C_{3} C_{5} R_{1}}{C_{1} C_{2} C_{3} C_{...,None,C_3*C_5*R_1/C_6,[],[Poly(s**3 + (C_1*C_2*C_3*R_1*R_3 + C_1*C_2*C_...,0,1
128,False,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",\frac{R_{1} R_{2}}{C_{1} C_{2} C_{6} R_{1} R_{...,None,R_1*R_2/C_6,[],"[Poly(s, s, domain='ZZ'), Poly(s**2 + (-C_1*R_...",0,2
129,True,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",\frac{R_{1} R_{2} R_{6}}{C_{1} C_{2} C_{6} R_{...,R_1*R_2*R_6/(R_1*R_5 - R_2*R_3 + R_2*R_5 + R_3...,R_1*R_2*R_6,[],"[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",0,2
130,False,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...",\frac{C_{5} R_{1} R_{2}}{C_{1} C_{2} C_{5} C_{...,None,C_5*R_1*R_2/C_6,[],[Poly(s**3 + (C_1*C_2*R_1*R_2*R_3 - C_1*C_5*R_...,0,1


In [154]:
results.experiment_name

'VLSI_CMMF_Automated_NA'

### HP-4th Order

In [160]:

def get_4th_order_hp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=4, printMessage=False)
    print(f"{count} candidates for 4th-order HP")

    output = []
    count = 0
    count_valid = 0 
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False
        # else:
        #     for exp in numer[0].as_dict():
        #         if exp != 4:
        #             valid = False


        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_hp = get_4th_order_hp(classifier)

df_hp = pd.DataFrame(output_hp)
df_hp.to_csv(f"Runs/{results.experiment_name}/4th_HP.csv")
df_hp

109 candidates for 4th-order HP


100%|██████████| 109/109 [00:19<00:00,  5.57it/s]


0 verified filters


,valid,zCombo,classification,k,numer,denom,num-factor-count,denom-factor-count
0,False,"(R_1, 1/(C_2*s), R_3/(C_3*R_3*s + 1), R_4 + 1/...","FilterClassification(valid=False, fType=X-INVA...",R_1/C_6,"[Poly(s + 1/(C_3*R_3), s, domain='ZZ(C_3,R_3)'...","[Poly(s, s, domain='ZZ'), Poly(s**3 + (C_2*C_3...",4,2
1,False,"(R_1, R_2 + 1/(C_2*s), R_3, R_4 + 1/(C_4*s), R...","FilterClassification(valid=False, fType=X-INVA...",-R_1/C_6,"[Poly(s + 1/(C_2*R_2), s, domain='ZZ(C_2,R_2)'...","[Poly(s, s, domain='ZZ'), Poly(s**3 + (-C_2*C_...",4,2
2,False,"(R_1, R_2 + 1/(C_2*s), 1/(C_3*s), R_4 + 1/(C_4...","FilterClassification(valid=False, fType=X-INVA...",C_2*C_3*C_4*C_6*R_1*R_4*R_5*R_6**2*(C_3*R_1 + ...,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_2*R_2)...","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",4,2
3,False,"(R_1, R_2 + 1/(C_2*s), R_3 + 1/(C_3*s), R_4 + ...","FilterClassification(valid=False, fType=X-INVA...",-C_3*R_1*R_6,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_2*R_2)...",[Poly(s**4 + (-C_2*C_3*C_4*R_1*R_4*R_5 + C_2*C...,4,1
4,False,"(R_1, R_2 + 1/(C_2*s), R_3 + 1/(C_3*s), R_4 + ...","FilterClassification(valid=False, fType=X-INVA...",-C_3*R_1/C_6,"[Poly(s + 1/(C_2*R_2), s, domain='ZZ(C_2,R_2)'...",[Poly(s**4 + (-C_2*C_3*C_4*R_1*R_4*R_5 + C_2*C...,4,1
...,...,...,...,...,...,...,...,...
104,False,"(R_1 + 1/(C_1*s), R_2 + 1/(C_2*s), R_3/(C_3*R_...","FilterClassification(valid=False, fType=X-INVA...",C_1*C_2*C_3*C_5**2*R_3*R_5*(R_1 + R_2)/(C_1*C_...,"[Poly(s + 1/(C_1*R_1), s, domain='ZZ(C_1,R_1)'...","[Poly(s, s, domain='ZZ'), Poly(s**3 + (C_1*C_2...",4,2
105,False,"(R_1 + 1/(C_1*s), R_2 + 1/(C_2*s), R_3/(C_3*R_...","FilterClassification(valid=False, fType=X-INVA...",C_1*C_2*R_3*R_5*(C_3*R_1 + C_3*R_2 - C_5*R_2)/...,"[Poly(s + 1/(C_1*R_1), s, domain='ZZ(C_1,R_1)'...","[Poly(s**2, s, domain='ZZ'), Poly(s**2 + (C_1*...",4,2
106,False,"(R_1 + 1/(C_1*s), R_2 + 1/(C_2*s), R_3/(C_3*R_...","FilterClassification(valid=False, fType=X-INVA...",C_1*C_2*C_6*R_3*R_5*R_6**2*(C_3*R_1 + C_3*R_2 ...,"[Poly(s + 1/(C_1*R_1), s, domain='ZZ(C_1,R_1)'...","[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_6*R_6)...",4,3
107,False,"(R_1 + 1/(C_1*s), R_2/(C_2*R_2*s + 1), R_3/(C_...","FilterClassification(valid=False, fType=X-INVA...",R_2/C_6,"[Poly(s + 1/(C_1*R_1), s, domain='ZZ(C_1,R_1)'...","[Poly(s, s, domain='ZZ'), Poly(s**3 + (C_1*C_2...",4,2


### BP-4th Order

In [159]:
classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)

def get_4th_order_bp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)
    print(f"{count} candidates for 4th-order BP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False

        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_bp = get_4th_order_bp(classifier)

df_bp = pd.DataFrame(output_bp)
df_bp.to_csv(f"Runs/{results.experiment_name}/4th_BP.csv")
df_bp

450 candidates for 4th-order BP


100%|██████████| 450/450 [01:27<00:00,  5.11it/s]


5 verified filters


,valid,zCombo,classification,k,numer,denom,num-factor-count,denom-factor-count
0,False,"(R_1, R_2, R_3 + 1/(C_3*s), 1/(C_4*s), R_5 + 1...","FilterClassification(valid=False, fType=X-INVA...",C_3*C_5*R_1*R_2*R_6,"[Poly(s**2, s, domain='ZZ')]","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",1,2
1,False,"(R_1, R_2, R_3 + 1/(C_3*s), R_4/(C_4*R_4*s + 1...","FilterClassification(valid=False, fType=X-INVA...",C_3*C_5*R_1*R_2*R_4*R_6,"[Poly(s**2, s, domain='ZZ')]","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",1,2
2,False,"(R_1, 1/(C_2*s), R_3, R_4 + 1/(C_4*s), R_5 + 1...","FilterClassification(valid=False, fType=X-INVA...",C_2*C_4*C_5**2*C_6*R_1*R_4*R_5*R_6**2*(R_1 + R...,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_4*R_4)...","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",2,2
3,False,"(R_1, 1/(C_2*s), 1/(C_3*s), R_4, R_5 + 1/(C_5*...","FilterClassification(valid=False, fType=X-INVA...",C_3*C_5*R_1*R_4*R_6,"[Poly(s**2, s, domain='ZZ')]","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",1,2
4,False,"(R_1, 1/(C_2*s), 1/(C_3*s), R_4 + 1/(C_4*s), R...","FilterClassification(valid=False, fType=X-INVA...",C_3*R_1*R_6,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_4*R_4)...","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",2,2
...,...,...,...,...,...,...,...,...
445,False,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...","FilterClassification(valid=False, fType=X-INVA...",C_3*C_5*R_1*R_2/C_6,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_6*R_6)...",[Poly(s**4 + (C_1*C_2*C_3*R_1*R_2*R_3 + C_1*C_...,2,1
446,False,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...","FilterClassification(valid=False, fType=X-INVA...",C_1*C_3**2*C_6*R_1**2*R_2**2*R_3*R_5*R_6**2*(C...,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_5*R_5)...","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",2,2
447,False,"(R_1/(C_1*R_1*s + 1), R_2/(C_2*R_2*s + 1), R_3...","FilterClassification(valid=False, fType=X-INVA...",C_5**2*C_6*R_1**2*R_2**2*R_3*R_5*R_6**2*(C_1*C...,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_3*R_3)...","[Poly(s + 1/(C_6*R_6), s, domain='ZZ(C_6,R_6)'...",2,2
448,False,"(R_1/(C_1*R_1*s + 1), oo, R_3 + 1/(C_3*s), oo,...","FilterClassification(valid=False, fType=X-INVA...",-C_3*R_1*R_6,"[Poly(s, s, domain='ZZ'), Poly(s + 1/(C_5*R_5)...","[Poly(s + 1/(C_1*R_1), s, domain='ZZ(C_1,R_1)'...",2,3


# Playground

In [ ]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function, variable, assumptions={}):
    """
    Decomposes the denominator of a transfer function into factors, 
    grouping strictly by the specified variable (e.g., `s`).

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        variable: The main variable to group by (e.g., `s`).
        assumptions: A dictionary of assumptions for symbolic variables.

    Returns:
        A dictionary where each key is a factor (grouped by the main variable),
        and the value contains its roots and properties.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Extract the denominator
    denominator = transfer_function.as_numer_denom()[1]
    
    # Ensure the denominator is treated as a polynomial in the given variable
    poly = Poly(denominator, variable)
    
    # Factorize strictly with respect to `variable`
    factored = factor(poly.as_expr())
    
    # Inspect each factor
    factors = factored.as_ordered_factors()
    results = {}
    for factor_expr in factors:
        # Solve for the roots of this factor
        roots = solveset(factor_expr, variable, domain=S.Complexes)
        
        # Analyze each root (e.g., real part)
        roots_info = []
        for root in roots:
            real_part = simplify(root.as_real_imag()[0])
            roots_info.append({
                "root": root,
                "real_part": real_part
            })
        
        # Store results for this factor
        results[str(factor_expr)] = {
            "factor": factor_expr,
            "roots": roots_info,
            "degree": Poly(factor_expr, variable).degree()
        }
    
    return results

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (L1 * s**3 + (R2 + R3) * s**2 + 1 / C1 * s + 1 / C2)

# Decompose and inspect the factors
results = decompose_tf(G, s)
for factor, info in results.items():
    print(f"Factor: {info['factor']}")
    print(f"Degree: {info['degree']}")
    print("Roots:")
    for root_info in info["roots"]:
        print(f"  - Root: {root_info['root']}, Real part: {root_info['real_part']}")
    print()


In [ ]:
results.keys()

In [51]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function):
    s = symbols("s")

    # Extract the numerator and denominator
    numerator, denominator = transfer_function.as_numer_denom()
 
    numerator = Poly(numerator, s)
    denominator = Poly(denominator, s)

    # Factorize strictly with respect to `variable`
    numerator = factor(numerator.as_expr())
    denominator = factor(denominator.as_expr())

    # Inspect each factor
    factors_numer = numerator.as_ordered_factors()
    factors_denom = denominator.as_ordered_factors()
    
    return factors_numer, factors_denom

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (((s+R3)*(s**2 + s*2*C2+ 2*C2**2)).expand())
G.expand()

# Decompose and inspect the factors
tup = decompose_tf(G)
numer = tup[0]
denom = tup[1]
